# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [11]:
# Do all imports and installs here
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import SparkSession


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [12]:
# Read in the data here
city_df = pd.read_csv('us-cities-demographics.csv',sep=';')
immigration_df = pd.read_csv('immigration_data_sample.csv').iloc[:,1:]
airport_df = pd.read_csv('airport-codes_csv.csv')

In [44]:
# Demography, examine the city based on size, and structure (Gender & Ethnicity composition)
# city_df.head()
city_demography = city_df[['City', 'State','State Code', 'Male Population', 'Female Population','Foreign-born','Number of Veterans','Race']]
city_demography.head()

,City,State,State Code,Male Population,Female Population,Foreign-born,Number of Veterans,Race
0,Silver Spring,Maryland,MD,40601.0,41862.0,30908.0,1562.0,Hispanic or Latino
1,Quincy,Massachusetts,MA,44129.0,49500.0,32935.0,4147.0,White
2,Hoover,Alabama,AL,38040.0,46799.0,8229.0,4819.0,Asian
3,Rancho Cucamonga,California,CA,88127.0,87105.0,33878.0,5821.0,Black or African-American
4,Newark,New Jersey,NJ,138040.0,143873.0,86253.0,5829.0,White


In [39]:
# statistics on city such as total population, and median age
# city_df.head()
city_stats = city_df[['City','State Code','Median Age','Average Household Size','Count']]
city_stats.head()

,City,State Code,Median Age,Average Household Size,Count
0,Silver Spring,MD,33.8,2.60,25924
1,Quincy,MA,41.0,2.39,58723
2,Hoover,AL,38.5,2.58,4759
3,Rancho Cucamonga,CA,34.5,3.18,24437
4,Newark,NJ,34.6,2.73,76402


In [14]:
immigration_df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [15]:
# make column names inteligible
immigration_df =immigration_df.rename(columns={'cicid':'cic_id','i94yr':'year','i94mon':'month','i94cit':'cit','i94port':'port','arrdate':'arrival_date','i94mode':'mode','i94addr':'address','depdate':'departure_date','i94bir':'age','i94visa':'visa','dtadfile':'date_logged','occup':'occupation','entdepa':'arrival_flag', \
                               'entdepd':'departure_flag','entdepu':'update_flag','matflag':'match_flag','biryear':'birth_year','dtaddto':'max_stay_date','insnum':'INS_number','admnnum':'admission_number','fltno':'flight_number', 'i94res':'res'}).head()
immigration_df.columns

Index(['cic_id', 'year', 'month', 'cit', 'res', 'port', 'arrival_date', 'mode',
       'address', 'departure_date', 'age', 'visa', 'count', 'date_logged',
       'visapost', 'occupation', 'arrival_flag', 'departure_flag',
       'update_flag', 'match_flag', 'birth_year', 'max_stay_date', 'gender',
       'INS_number', 'airline', 'admnum', 'flight_number', 'visatype'],
      dtype='object')

In [16]:
# build fact table from immigration_df 

immigration_fact = immigration_df[['cic_id', 'year', 'month','arrival_date','departure_date','mode','visatype']]
immigration_fact.head()

,cic_id,year,month,arrival_date,departure_date,mode,visatype
0,4084316.0,2016.0,4.0,20566.0,20573.0,1.0,WT
1,4422636.0,2016.0,4.0,20567.0,20568.0,1.0,B2
2,1195600.0,2016.0,4.0,20551.0,20571.0,1.0,WT
3,5291768.0,2016.0,4.0,20572.0,20581.0,1.0,B2
4,985523.0,2016.0,4.0,20550.0,20553.0,3.0,WT


In [29]:
airport_df = airport_df[(airport_df.iso_country == 'US')]
airport_df = airport_df[~(airport_df.type =='closed')]
airport_dim = airport_df[['ident', 'type', 'name', 'continent', 'gps_code', 'iata_code','local_code']]
airport_dim.head()

,ident,type,name,continent,gps_code,iata_code,local_code
0,00A,heliport,Total Rf Heliport,NaN,00A,NaN,00A
1,00AA,small_airport,Aero B Ranch Airport,NaN,00AA,NaN,00AA
2,00AK,small_airport,Lowell Field,NaN,00AK,NaN,00AK
3,00AL,small_airport,Epps Airpark,NaN,00AL,NaN,00AL
5,00AS,small_airport,Fulton Airport,NaN,00AS,NaN,00AS


In [31]:
airport_stats = airport_df[['ident', 'elevation_ft','coordinates']]
airport_stats.head()

,ident,elevation_ft,coordinates
0,00A,11.0,"-74.93360137939453, 40.07080078125"
1,00AA,3435.0,"-101.473911, 38.704022"
2,00AK,450.0,"-151.695999146, 59.94919968"
3,00AL,820.0,"-86.77030181884766, 34.86479949951172"
5,00AS,1100.0,"-97.8180194, 34.9428028"


In [46]:
# Temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temp_df = pd.read_csv(fname)
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [1]:

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [2]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [52]:
# Performing cleaning tasks here

def sas_to_date(date):
    # 1. sas dates to python datetime
    """
        datetime.strptime('2016-01-01', '%Y-%m-%d') - timedelta(20566) gives a date between the year 1959 and 1960 
        working with the date 1960-01-01 gives a more accurate date that reflects the arrival and departure dates
    """
    return (datetime.strptime('1960-01-01', '%Y-%m-%d') + timedelta(date)).strftime('%Y-%m-%d')

immigration_fact['arrival_date'] = immigration_fact.arrival_date.apply(lambda x: sas_to_date(x))
immigration_fact['departure_date'] = immigration_fact.departure_date.apply(lambda x: sas_to_date(x))

immigration_fact.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,cic_id,year,month,arrival_date,departure_date,mode,visatype
0,4084316.0,2016.0,4.0,2016-04-22,2016-04-29,1.0,WT
1,4422636.0,2016.0,4.0,2016-04-23,2016-04-24,1.0,B2
2,1195600.0,2016.0,4.0,2016-04-07,2016-04-27,1.0,WT
3,5291768.0,2016.0,4.0,2016-04-28,2016-05-07,1.0,B2
4,985523.0,2016.0,4.0,2016-04-06,2016-04-09,3.0,WT


In [64]:
#check missing data
us_array = ['usa','united states of america', 'us']
temp_df['Country'] = temp_df.Country.apply(lambda x: x.lower())
temp_df.Country.isin(us_array).unique()

array([False], dtype=bool)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.